This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

NqNSzHiNrV1p3VDS5kC9


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import os,sys
import json
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [4]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
base_url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json"
search_params = {'api_key':API_KEY, 'start_date': '2022-11-01', 'end_date':'2022-11-01'}
req_small = requests.get(base_url, params=search_params)
print (req_small.status_code)

200


In [8]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data_small = json.loads(req_small.content.decode('utf-8'))
data_small

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2022-11-01',
  'end_date': '2020-12-01',
  'frequency': 'daily',
  'data': [],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [9]:
# Task1 
search_params.update(start_date='2017-01-01', end_date='2017-12-31')
req = requests.get(base_url, params=search_params)
req.status_code

200

In [15]:
# Task2
raw = json.loads(req.content.decode('utf-8'))
dataset = []
for data in raw['dataset_data']['data'] : 
    dataset.append(dict(zip(raw['dataset_data']['column_names'], data)))
print (len(dataset))
dataset[0]

255


{'Date': '2017-12-29',
 'Open': 51.76,
 'High': 51.94,
 'Low': 51.45,
 'Close': 51.76,
 'Change': None,
 'Traded Volume': 34640.0,
 'Turnover': 1792304.0,
 'Last Price of the Day': None,
 'Daily Traded Units': None,
 'Daily Turnover': None}

In [24]:
# 3.Calculate what the highest and lowest opening prices were for the stock in this period.
import numpy as np
data_has_open = [data for data in dataset if data['Open'] is not None]
open_sorted = sorted(data_has_open, key=lambda data: data['Open'], reverse=True)
print ('Highest open is %s, occurred on %s' %(open_sorted[0]['Open'], open_sorted[0]['Date']))
print ('Lowest open is %s, occurred on %s' %(open_sorted[-1]['Open'], open_sorted[-1]['Date']))

Highest open is 53.11, occurred on 2017-12-14
Lowest open is 34.0, occurred on 2017-01-24


In [31]:
# 4.What was the largest change in any one day (based on High and Low price)?
for data in dataset : 
    diff = float(data['High']) - float(data['Low'])
    data.update(Diff = str(diff))
diff_sorted = sorted(dataset, key = lambda data: data['Diff'], reverse = True)
print ('The largest change is %s, occurred on %s' %(diff_sorted[0]['Diff'], diff_sorted[0]['Date']))

The largest change is 2.8100000000000023, occurred on 2017-05-11


In [35]:
# 5. What was the largest change between any two days (based on Closing Price)?
date_sorted = sorted(dataset, key = lambda data: data['Date'])
change_list = []
i = 0 
while i < len(date_sorted)-1 : 
    change = abs(float(date_sorted[i]['Close']) - float(date_sorted[i+1]['Close']))
    change_list.append([date_sorted[i], date_sorted[i+1], {'Close_change':str(change)}])
    i += 1 
change_sorted = sorted(change_list, key = lambda data: data[2]['Close_change'], reverse = True)
print ('The close price had changed the most in between %s and %s, the difference is %s' %(change_sorted[0][0]['Date'], change_sorted[0][1]['Date'], change_sorted[0][2]['Close_change']))
    

The close price had changed the most in between 2017-08-08 and 2017-08-09, the difference is 2.559999999999995


In [39]:
# 6. What was the average daily trading volume during this year?
tradeVol_list = [float(data['Traded Volume']) for data in dataset if data['Traded Volume'] is not None]
print ('The average daily trading volume in 2017 is %f' % (sum(tradeVol_list)/len(tradeVol_list)))

The average daily trading volume in 2017 is 89124.337255


In [53]:
# 7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
def get_median(alist) : 
    # make sure the items in alist are numbers
    new = [float(x) for x in alist]        
    new.sort()
    N = len(new)
    if N % 2 == 0 : 
        i = int(N/2-1)
        median = (new[i] + new[i+1])/2
    else : 
        i = int((N-1)/2)
        median = new[i]
    return str(median)
print ('The median trading volume in 2017 is %s' % get_median(tradeVol_list))

The median trading volume in 2017 is 76286.0
